<a href="https://colab.research.google.com/github/aknip/Streamlit-Gradio/blob/main/helper_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Promptlib

In [ ]:
# myprompt = "The given text is delimited by triple backticks. Summarize the current text to succint and clear bullet points of its contents.
#          The length of the summary must be 200 words maximum."
# myprompt = "The given text is delimited by triple backticks.
#           Summarize the current text to a maximum of 15 succint and clear bullet points of its contents."
# myprompt = myprompt + "The maximum number of words should be 300 words in total. "
# myprompt = myprompt + "Write everything in German language. " # this is optional !
# myprompt = myprompt + "```" + input_text + "```"


# Promptlib
# Todo: promptlib not as global

def testfunc(a,b):
  result = a+b
  return result

promptlib_global = {
    'summary-shortened': {
      'description': '',
      'category': 'summarize',
      '1': {
          'note': 'Gleichzeitiges Einfügen/Formatieren von Headlines funktioniert nicht, lässt GPT alles umformulieren\
           ("Format headings and subheadings in markdown syntax.").\
           Test S4/HANA Kap. 4: Input 12.300 Wörter, GPT-3.5 Output: 4.500 Wörter (36%), GPT-4 Output: 3.400 Wörter (27%)',
          "prompt": "Paraphrase the following text and shorten each paragraph of the paraphrased text to maxiumum the half of its original length by focussing\
            on the key aspects. Write in the style and tone of a professional business analyst and use passive form. Be detailed and precise\
            and use every content detail of the text. Note that there may be typographical or grammatical errors in the passages due to the\
            text being fetched from a transcript of a talk or a speech.\
            {language_text} {length_text} Text:```{input_text}``` {toc_text}",
          "lang-de": "Write everything in German language.",
          "lang-en": "Write everything in English language.",
          "lang-same": "If the following text in triple backticks is in German language write your text in German language, too.",
          'gpt-3.5-turbo': {
              "input-text-max": 1200,
              "length-max-fix": "The length of the full text must be 1200 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'gpt-4': {
              "input-text-max": 2400,
              "length-max-fix": "The length of the full text must be 2400 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'default-model': {
              "input-text-max": 500,
              "length-max-fix": "The length of the full text must be 500 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          }
      }
    },
    'summary-multiple-articles': {
      'description': '',
      'category': 'summarize',
      '1': {
          'note': 'summarize each headline / paragraph',
          "prompt": "The given text is delimited by triple backticks. It is a newspaper text with several articles.\
            Summarize each article with its headline and succint and clear bullet points of its contents.\
            Format everything in markdown syntax. {language_text} {length_text} Text:```{input_text}``` {toc_text}",
          "lang-de": "Write everything in German language.",
          "lang-en": "Write everything in English language.",
          "lang-same": "If the following text in triple backticks is in German language write your text in German language, too.",
          'gpt-3.5-turbo': {
              "input-text-max": 1200,
              "length-max-fix": "The length of the full text must be 1200 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'gpt-4': {
              "input-text-max": 2400,
              "length-max-fix": "The length of the full text must be 2400 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'default-model': {
              "input-text-max": 500,
              "length-max-fix": "The length of the full text must be 500 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          }
      }
    },
    'summary-bullets': {
      'description': 'Max word length: 1100 for EN, 900 for DE. approx. 1 min processing time Example: Full book: 113 parts á 1000 words takes 15 min.',
      'category': 'summarize',
      '1': {
          'note': 'summarize to bullets v1',
          "prompt": 'The given text is delimited by triple backticks. Summarize the current text to succint and clear bullet points\
            of its contents. {language_text} {length_text} Text:```{input_text}``` {toc_text}',
          "lang-de": "Write everything in German language.",
          "lang-en": "Write everything in English language.",
          "lang-same": "If the following text in triple backticks is in German language write your text in German language, too.",
          'gpt-3.5-turbo': {
              "input-text-max": 1200,
              "length-max-fix": "The length of the summary must be 300 words maximum.",
              "length-max-dyn": "The length of the summary must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'gpt-4': {
              "input-text-max": 2400,
              "length-max-fix": "The length of the summary must be 300 words maximum.",
              "length-max-dyn": "The length of the summary must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'default-model': {
              "input-text-max": 500,
              "length-max-fix": "The length of the summary must be 300 words maximum.",
              "length-max-dyn": "The length of the summary must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          }
      },
      '2': {
          'note': 'summarize to bullets v2',
          "prompt": 'The given text is delimited by triple backticks. Summarize the current text to a maximum of 15 succint\
            and clear bullet points of its contents. {language_text} {length_text} Text:```{input_text}``` {toc_text}',
          "lang-de": "Write everything in German language.",
          "lang-en": "Write everything in English language.",
          "lang-same": "If the following text in triple backticks is in German language write your text in German language, too.",
          'gpt-3.5-turbo': {
              "input-text-max": 1200,
              "length-max-fix": "The length of the summary must be 300 words maximum.",
              "length-max-dyn": "The length of the summary must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'gpt-4': {
              "input-text-max": 2400,
              "length-max-fix": "The length of the summary must be 300 words maximum.",
              "length-max-dyn": "The length of the summary must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'default-model': {
              "input-text-max": 500,
              "length-max-fix": "The length of the summary must be 300 words maximum.",
              "length-max-dyn": "The length of the summary must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          }
      }
    },
    'create-subheadlines': {
      'description': '',
      'category': 'structure',
      '1': {
          'note': 'create subheadlien, keep text the same',
          "prompt": "The given text is delimited by triple backticks. Divide the text into 3 paragraphs of approximately\
           equal length according to the meaning of the content and insert subheadings for the paragraphs.\
           Please respond with the full original text including the inserted subheadings.\
           Format the subheadings in markdown syntax. {language_text} {length_text} Text:```{input_text}``` {toc_text}",
          "lang-de": "Write everything in German language.",
          "lang-en": "Write everything in English language.",
          "lang-same": "Write in the language of the original text.",
          'gpt-3.5-turbo': {
              "input-text-max": 1200,
              "length-max-fix": "The length of the full text must be 1200 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'gpt-4': {
              "input-text-max": 2400,
              "length-max-fix": "The length of the full text must be 2400 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'default-model': {
              "input-text-max": 500,
              "length-max-fix": "The length of the full text must be 500 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          }
      }
    },
    'summary-3-sentences': {
      'description': '',
      'category': 'summarize',
      '1': {
          'note': 'create subheadlien, keep text the same',
          "prompt": "The given text is delimited by triple backticks. Summarize the text into three\
           sentences. {language_text} {length_text} Text:```{input_text}``` {toc_text}",
          "lang-de": "Write everything in German language.",
          "lang-en": "Write everything in English language.",
          "lang-same": "Write in the language of the original text.",
          'gpt-3.5-turbo': {
              "input-text-max": 1200,
              "length-max-fix": "The length of the full text must be 300 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'gpt-4': {
              "input-text-max": 2400,
              "length-max-fix": "The length of the full text must be 300 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          },
          'default-model': {
              "input-text-max": 500,
              "length-max-fix": "The length of the full text must be 300 words maximum.",
              "length-max-dyn": "The length of the full text must be {{length_calc(\"{input_text}\", {max_len})}} words maximum."
          }
      }
    }
  }